In [ ]:
!pip install simple_diarizer

In [16]:
from simple_diarizer.diarizer import Diarizer

In [ ]:
!pip install pytube
!pip install -U openai-whisper
!pip install pydub

In [19]:
from pytube import YouTube
import whisper
from pydub import AudioSegment

In [ ]:
model = whisper.load_model("medium")
diarization = Diarizer(embed_model='xvec', cluster_method='sc')

In [22]:
def get_full_answer(audio_file_name) -> str:
  prepared_audio = AudioSegment.from_file(audio_file_name, format="mp4")
  print("аудио было приготовлено")

  def get_slice_audio(audio_file_name, begin: int, end: int, trimmed_audio_name: str, format_audio):
    trimmed_audio = prepared_audio[begin: end]
    trimmed_audio.export(trimmed_audio_name, format=format_audio)

  def get_transcription(audio_file_name) -> str:
    audio = whisper.load_audio(audio_file_name)
    audio = whisper.pad_or_trim(audio)

    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)

    return result.text

  segments = diarization.diarize(audio_file_name, num_speakers=4)
  print("сегментировано")
  cnt_segments = len(segments)
  answer = []
  i, j = 0, 0
  while j < cnt_segments:
    while j < cnt_segments and segments[j]["label"] == segments[i]["label"]:
      j += 1
    j -= 1

    new_audio = "tmp.wav"
    get_slice_audio(audio_file_name, segments[i]["start"] * 1000, segments[j]["end"] * 1000, new_audio, 'wav')
    answer.append([segments[j]["label"], int(segments[i]["start"]), int(segments[j]["end"]), get_transcription(new_audio)])
    i = j = j + 1
    print(f"обработано {i} семгментов")
  return str(answer)

In [ ]:
!pip install pyTelegramBotAPI

In [40]:
import telebot
from telebot import types

# Замените 'YOUR_BOT_TOKEN' на фактический токен вашего Telegram-бота
bot = telebot.TeleBot('6645231772:AAGfVSHptZrRZurqejzmt7DtA4x-6WxQeJU')

In [ ]:
@bot.message_handler(commands=['start'])
def handle_start(message):
  bot.send_message(message.chat.id, "Добро пожаловать в бот транскрибации! Отправьте мне видео или аудиосообщение для транскрибации.")

def send_answer_to(message):
  print("получено аудио")
  final_answer = "empty"
  if message.content_type == "voice":
    audio = message.voice
    file_id = audio.file_id
    file_info = bot.get_file(message.voice.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    with open('audio.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
  else:
    audio = message.audio
    file_id = audio.file_id

    file = bot.get_file(file_id)
    print("скачивается аудио")
    file.download("audio.ogg")
  print("скачано аудио")

  sound = AudioSegment.from_ogg("audio.ogg")
  sound.export("audio.wav", format="wav")
  final_answer = get_full_answer("audio.wav")
  bot.send_message(message.chat.id, final_answer)

@bot.message_handler(content_types=['audio'])
def handle_media_message(message):
  send_answer_to(message)

@bot.message_handler(content_types=['voice'])
def handle_media_message(message):
  send_answer_to(message)

@bot.message_handler(content_types=['video'])
def handle_media_message(message):
  transcription = "it's video"

  bot.send_message(message.chat.id, transcription)

@bot.message_handler(content_types=['photo'])
def handle_picture(message):
  print('dlafjsdlfj')
  bot.send_message(message.chat.id, "You send picture")


if __name__ == '__main__':
    bot.polling(none_stop=True)


получено аудио
скачано аудио
аудио было приготовлено
Converting audio file to single channel WAV using ffmpeg...
Running VAD...
Splitting by silence found 2 utterances
Extracting embeddings...


Utterances:   0%|          | 0/2 [00:00<?, ?it/s]

Clustering to 4 speakers...


/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_spectral_embedding.py:248: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  adjacency = check_symmetric(adjacency)
/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


Cleaning up output...
Done!
сегментировано
обработано 1 семгментов
обработано 2 семгментов
обработано 3 семгментов
обработано 4 семгментов
